In [1]:
import csv
from glob import glob
import pandas as pd
import numpy as np
import sys

In [2]:
# fixed files
scv2_ref = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/scv2.fasta'
gff = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/sc2_noCDS.gff'
bed = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/nCoV-2019.primer.bed'
adapters = '/shared/software/bbmap/v38.78/resources/adapters.fa'
phiX = '/shared/software/bbmap/v38.78/resources/phix174_ill.ref.fa.gz'
swift_masterfile = '~/bin/swift_sarscov2_v2_masterfile.txt'

bb = '/shared/software/bin/bbduk.sh'
ivar = 'singularity run docker://quay.io/biocontainers/ivar:1.3.1--h089eab3_0 ivar'
primerclip = 'singularity exec docker://quay.io/biocontainers/primerclip:0.3.8--h9ee0642_1 primerclip'
freebayes = 'singularity run https://depot.galaxyproject.org/singularity/freebayes:1.3.5--py39hba5d119_3 freebayes'
picard = 'singularity run https://depot.galaxyproject.org/singularity/picard:2.25.4--hdfd78af_0 picard'
#mutect2 = '/shared/software/bin/gatk Mutect2'
mutect2 = 'singularity run https://depot.galaxyproject.org/singularity/gatk4:4.2.0.0--hdfd78af_1 gatk Mutect2'

In [6]:
# # 3/20/21
# rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_033021'
# trimmedreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_033021/trimmed_reads'
# mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_033021'

# rawreads_sw = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_swift_033021'
# trimmedreads_sw = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_swift_033021/trimmed_reads'
# mapping_sw = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_swift_033021'


# reads_artic_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_march2021/artic_files.tsv', sep='\t')
# reads_swift_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_march2021/swift_files.tsv', sep='\t')

In [7]:
# # 5/7/21
# rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_050721'
# trimmedreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_050721/trimmed_reads'
# mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_050721'

# outdir = '/Users/rosekantor/work/wbe_sequence_analysis/qb3_may2021'
# # no sample names available yet, so starting processing with just a list of read names, added names2samples afterward
# reads_artic_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_may2021/reads_list.txt', names=['read1'])
# names2samples = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_may2021/names2samples.csv')

# # create columns for read2 and sample_num
# reads_artic_df['read2'] = reads_artic_df.read1.str.replace(r'_R1_', '_R2_')
# reads_artic_df['Name'] = reads_artic_df.read1.str.split('_', expand=True)[0]

# reads_artic_df = reads_artic_df.merge(names2samples, on='Name')
# reads_artic_df['read2'] = reads_artic_df.read1.str.replace(r'_R1_', '_R2_')
# reads_artic_df['sample'] = reads_artic_df.read1.str.split('_', expand=True)[0]

In [29]:
# # 5/26/21
# rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_052621'
# trimmedreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_052621/trimmed_reads'
# mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_052621'
# outdir = '/Users/rosekantor/work/wbe_sequence_analysis/qb3_artic_052621'
# datadir = '/Users/rosekantor/data/wbe_scv/qb3_artic_052621/'

# reads_artic_df = pd.read_csv('/Users/rosekantor/data/wbe_scv/qb3_artic_052621/reads_list.txt', names=['read1'])
# reads_artic_df = reads_artic_df[~reads_artic_df.read1.str.contains('Undetermined')]

In [12]:
# # 6/29/21
# rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_062921'
# trimmedreads = f'{rawreads}/trimmed_reads'
# mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_062921'
# outdir = '/Users/rosekantor/work/wbe_sequence_analysis/qb3_artic_062921'
# datadir = '/Users/rosekantor/data/wbe_scv/qb3_artic_062921/'

# reads_artic_df = pd.read_csv(f'{datadir}/reads_list.txt', names=['read1'])
# reads_artic_df = reads_artic_df[~reads_artic_df.read1.str.contains('Undetermined')]

# #create columns for read2 and sample_num

# # for 6/29 need to keep the treatment in the name to distinguish them
# reads_artic_df['read2'] = reads_artic_df.read1.str.replace(r'_R1_', '_R2_')
# reads_artic_df['treatment'] = reads_artic_df.read1.str.split('_', expand=True)[0]
# reads_artic_df['sample'] = reads_artic_df.read1.str.extract(r'(.+)_S\d+_L001_R._001\.fastq\.gz')
# reads_artic_df['sample_id'] = reads_artic_df.read1.str.extract(r'.+_(\w+_\w+_\w+_\d+_\d)_.+')

# # fill in the name of the sample we ran the isopropanol precipitation on
# # (which was actually all 3 reps of K_K_INF from 5/30 and 5/31)
# reads_artic_df.loc[reads_artic_df['sample'].str.contains('IPprep'), 'sample_id'] = 'K_K_INF_053021_1'

# #reads_artic_df['sample'] = reads_artic_df.read1.str.extract(r'(\w+_\w+_\w+_\d+_\d)_.+') # original
# reads_artic_df

In [7]:
# 7/30/21
rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_073021'
trimmedreads = f'{rawreads}/trimmed_reads'
mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_073021'
outdir = '/Users/rosekantor/work/wbe_sequence_analysis/qb3_artic_073021'
datadir = '/Users/rosekantor/data/wbe_scv/qb3_artic_073021'

reads_artic_df = pd.read_csv(f'{datadir}/reads_list.txt', names=['read1'])
reads_artic_df = reads_artic_df[~reads_artic_df.read1.str.contains('Undetermined')]

# create columns for read2 and sample_num

# for 7/30, add the treatment in the name to distinguish them
reads_artic_df['read2'] = reads_artic_df.read1.str.replace(r'_R1_', '_R2_')
reads_artic_df['sample'] = reads_artic_df.read1.str.extract(r'(.+)_S\d+_L001_R._001\.fastq\.gz')
reads_artic_df['sample_id'] = reads_artic_df.read1.str.extract(r'(\w+_\w+_\w+_\d+_\d).*')
reads_artic_df['treatment'] = 'std'
reads_artic_df.loc[reads_artic_df['sample'].str.contains('4A'), 'treatment'] = 'amicon'
reads_artic_df.loc[reads_artic_df['sample'].str.contains('4N'), 'treatment'] = 'nanotrap'

reads_artic_df

In [24]:
# 8/7/21
rawreads = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/raw/qb3_artic_080721'
trimmedreads = f'{rawreads}/trimmed_reads'
mapping_art = '/groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/qb3_artic_080721'
outdir = '/Users/rosekantor/work/wbe_sequence_analysis/qb3_artic_080721'
datadir = '/Users/rosekantor/data/wbe_scv/qb3_artic_080721'

reads_artic_df = pd.read_csv(f'{datadir}/reads_list.txt', names=['read1'])
reads_artic_df = reads_artic_df[~reads_artic_df.read1.str.contains('Undetermined')]

# create columns for read2 and sample_num

# for 7/30, add the treatment in the name to distinguish them
reads_artic_df['read2'] = reads_artic_df.read1.str.replace(r'_R1_', '_R2_')
reads_artic_df['sample'] = reads_artic_df.read1.str.extract(r'(.+)_S\d+_L001_R._001\.fastq\.gz')
reads_artic_df['sample_id'] = reads_artic_df.read1.str.extract(r'(\w+_\w+_\w+_\d+_\d).*')
reads_artic_df['treatment'] = 'std'

reads_artic_df

,read1,read2,sample,sample_id,treatment
0,C_C_S68_071421_1_S13_L001_R1_001.fastq.gz,C_C_S68_071421_1_S13_L001_R2_001.fastq.gz,C_C_S68_071421_1,C_C_S68_071421_1,std
1,C_C_S68_071421_2_S14_L001_R1_001.fastq.gz,C_C_S68_071421_2_S14_L001_R2_001.fastq.gz,C_C_S68_071421_2,C_C_S68_071421_2,std
2,D_OSP_INF_071121_3_S11_L001_R1_001.fastq.gz,D_OSP_INF_071121_3_S11_L001_R2_001.fastq.gz,D_OSP_INF_071121_3,D_OSP_INF_071121_3,std
3,D_OSP_INF_071421_1_S12_L001_R1_001.fastq.gz,D_OSP_INF_071421_1_S12_L001_R2_001.fastq.gz,D_OSP_INF_071421_1,D_OSP_INF_071421_1,std
4,D_SEP_NF_071521_1_S3_L001_R1_001.fastq.gz,D_SEP_NF_071521_1_S3_L001_R2_001.fastq.gz,D_SEP_NF_071521_1,D_SEP_NF_071521_1,std
5,D_SEP_NF_071521_3_S4_L001_R1_001.fastq.gz,D_SEP_NF_071521_3_S4_L001_R2_001.fastq.gz,D_SEP_NF_071521_3,D_SEP_NF_071521_3,std
6,G_G_INF_071321_1_S5_L001_R1_001.fastq.gz,G_G_INF_071321_1_S5_L001_R2_001.fastq.gz,G_G_INF_071321_1,G_G_INF_071321_1,std
7,G_G_INF_071321_2_S6_L001_R1_001.fastq.gz,G_G_INF_071321_2_S6_L001_R2_001.fastq.gz,G_G_INF_071321_2,G_G_INF_071321_2,std
8,G_G_MCS_071321_1_S1_L001_R1_001.fastq.gz,G_G_MCS_071321_1_S1_L001_R2_001.fastq.gz,G_G_MCS_071321_1,G_G_MCS_071321_1,std
9,G_G_MCS_071321_2_S2_L001_R1_001.fastq.gz,G_G_MCS_071321_2_S2_L001_R2_001.fastq.gz,G_G_MCS_071321_2,G_G_MCS_071321_2,std


In [25]:
reads_artic_df.to_csv(f'{datadir}/samples_df.csv', index=False)

# Pipeline - ARTIC

## Make read trimming commands to remove adapters and low quality regions

In [26]:
cmds = []
for row in reads_artic_df.itertuples():
    r1 = row.read1
    r2 = row.read2
    s = row.sample
    trimR = f'{bb} -Xmx5g k=23 mink=11 hdist=1 tbo tpe t=2 ref={adapters} ktrim=r ftm=5 -in1={rawreads}/{r1} -in2={rawreads}/{r2} -out1={rawreads}/{s}.1.noadapt.fastq.gz -out2={rawreads}/{s}.2.noadapt.fastq.gz'
    #filtphiX = f'{bb} -Xmx5g t=2 k=31 hdist=1 -in1={rawreads}/{s}.1.noadapt.fastq.gz -in2={rawreads}/{s}.2.noadapt.fastq.gz out1={rawreads}/{s}.1.nophix.fastq.gz out2={rawreads}/{s}.2.nophix.fastq.gz ref={phiX} stats={s}_stats_phix.txt'
    qtrim = f'sickle pe -l 75 -f {rawreads}/{s}.1.noadapt.fastq.gz -r {rawreads}/{s}.2.noadapt.fastq.gz -t sanger -o {trimmedreads}/{s}.PE.1.fastq -p {trimmedreads}/{s}.PE.2.fastq -s {trimmedreads}/{s}.SR.fastq'
    clean = f'rm {rawreads}/{s}.1.noadapt.fastq.gz {rawreads}/{s}.2.noadapt.fastq.gz {trimmedreads}/{s}.SR.fastq'
    #qc = f'{fastqc} -t 2 {s}.PE.1.fastq {s}.PE.2.fastq'
    gz1 = f'pigz -p 2 {trimmedreads}/{s}.PE.1.fastq'
    gz2 = f'pigz -p 2 {trimmedreads}/{s}.PE.2.fastq'
    cmd = [trimR, qtrim, clean, gz1, gz2]
    cmd = "; ".join(cmd)
    cmds.append([s, cmd])

prep_artic_df = pd.DataFrame(cmds, columns=[
                        'sample',
                        'read_trimming'
                    ])

## Make iVar pipeline for ARTIC data
Note: not in mapping subdirectory

In [30]:
cmds = []
for s in reads_artic_df['sample']:
    mapping = f'bowtie2 -x {scv2_ref} -p 48 -1 {trimmedreads}/{s}.PE.1.fastq.gz -2 {trimmedreads}/{s}.PE.2.fastq.gz 2>{mapping_art}/{s}.log | sambam > {mapping_art}/{s}.bam'
    map_qcmd = f'sbatch --wrap "{mapping}"'
    
    primer_trimming = f'{ivar} trim -e -q 0 -i {mapping_art}/{s}.bam  -b {bed} -p {mapping_art}/{s}.primer_trimmed 2>&1 | tee -a {mapping_art}/{s}.ivar_trim.log'
    sort_index_bam = f'samtools sort {mapping_art}/{s}.primer_trimmed.bam > {mapping_art}/{s}.primer_trimmed.sorted.bam && samtools index {mapping_art}/{s}.primer_trimmed.sorted.bam'
    samtools_depth = f'samtools depth {mapping_art}/{s}.primer_trimmed.sorted.bam > {mapping_art}/{s}.depth.txt'
    mpileup_variants = f'samtools mpileup -aa -A  -B -Q 0 {mapping_art}/{s}.primer_trimmed.sorted.bam | {ivar} variants -p {mapping_art}/{s}.ivar -q 20 -t 0.03 -r {scv2_ref} -g {gff}'
    freebayes_variants = f'{freebayes} -f {scv2_ref} -b {mapping_art}/{s}.primer_trimmed.sorted.bam -F 0.05 -C 2 --pooled-continuous > {mapping_art}/{s}.freebayes.vcf'
    # gatk requires readgroup info to be added to the bam files, so we make this up
    picard_readgrp = f'{picard} AddOrReplaceReadGroups I={mapping_art}/{s}.primer_trimmed.sorted.bam O={mapping_art}/{s}.primer_trimmed.sorted.rg.bam RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM={s}'
    # index again... sigh
    index_bam = f'samtools index {mapping_art}/{s}.primer_trimmed.sorted.rg.bam'
    gatk_variants = f'{mutect2} --input {mapping_art}/{s}.primer_trimmed.sorted.rg.bam --reference {scv2_ref} --output {mapping_art}/{s}.gatk.vcf'
    
    cmds.append([s, map_qcmd, primer_trimming, sort_index_bam, samtools_depth,
                 mpileup_variants, freebayes_variants, picard_readgrp, index_bam, gatk_variants])

variant_df = pd.DataFrame(cmds, columns=[
                        'sample',
                        'mapping',
                        'primer_trimming',
                        'sort_index_bam',
                        'samtools_depth',
                        'mpileup_variants',
                        'freebayes_variants',
                        'picard_readgrp',
                        'index_bam',
                        'gatk_variants'
                    ])
cmds_artic_df = prep_artic_df.merge(variant_df, on='sample')

In [31]:
cmds_artic_df.head()

,sample,read_trimming,mapping,primer_trimming,sort_index_bam,samtools_depth,mpileup_variants,freebayes_variants,picard_readgrp,index_bam,gatk_variants
0,C_C_S68_071421_1,/shared/software/bin/bbduk.sh -Xmx5g k=23 mink...,bowtie2 -x /groups/banfield/projects/industria...,singularity run docker://quay.io/biocontainers...,samtools sort /groups/banfield/projects/indust...,samtools depth /groups/banfield/projects/indus...,samtools mpileup -aa -A -B -Q 0 /groups/banfi...,singularity run https://depot.galaxyproject.or...,singularity run https://depot.galaxyproject.or...,samtools index /groups/banfield/projects/indus...,singularity run https://depot.galaxyproject.or...
1,C_C_S68_071421_2,/shared/software/bin/bbduk.sh -Xmx5g k=23 mink...,bowtie2 -x /groups/banfield/projects/industria...,singularity run docker://quay.io/biocontainers...,samtools sort /groups/banfield/projects/indust...,samtools depth /groups/banfield/projects/indus...,samtools mpileup -aa -A -B -Q 0 /groups/banfi...,singularity run https://depot.galaxyproject.or...,singularity run https://depot.galaxyproject.or...,samtools index /groups/banfield/projects/indus...,singularity run https://depot.galaxyproject.or...
2,D_OSP_INF_071121_3,/shared/software/bin/bbduk.sh -Xmx5g k=23 mink...,bowtie2 -x /groups/banfield/projects/industria...,singularity run docker://quay.io/biocontainers...,samtools sort /groups/banfield/projects/indust...,samtools depth /groups/banfield/projects/indus...,samtools mpileup -aa -A -B -Q 0 /groups/banfi...,singularity run https://depot.galaxyproject.or...,singularity run https://depot.galaxyproject.or...,samtools index /groups/banfield/projects/indus...,singularity run https://depot.galaxyproject.or...
3,D_OSP_INF_071421_1,/shared/software/bin/bbduk.sh -Xmx5g k=23 mink...,bowtie2 -x /groups/banfield/projects/industria...,singularity run docker://quay.io/biocontainers...,samtools sort /groups/banfield/projects/indust...,samtools depth /groups/banfield/projects/indus...,samtools mpileup -aa -A -B -Q 0 /groups/banfi...,singularity run https://depot.galaxyproject.or...,singularity run https://depot.galaxyproject.or...,samtools index /groups/banfield/projects/indus...,singularity run https://depot.galaxyproject.or...
4,D_SEP_NF_071521_1,/shared/software/bin/bbduk.sh -Xmx5g k=23 mink...,bowtie2 -x /groups/banfield/projects/industria...,singularity run docker://quay.io/biocontainers...,samtools sort /groups/banfield/projects/indust...,samtools depth /groups/banfield/projects/indus...,samtools mpileup -aa -A -B -Q 0 /groups/banfi...,singularity run https://depot.galaxyproject.or...,singularity run https://depot.galaxyproject.or...,samtools index /groups/banfield/projects/indus...,singularity run https://depot.galaxyproject.or...


In [32]:
cmds_artic_df.read_trimming.to_csv(f'{outdir}/read_trimming_artic.sh', index=False, header=False)
cmds_artic_df.mapping.to_csv(f'{outdir}/mapping_artic.sh', index=False, header=False, quoting=csv.QUOTE_NONE)
cmds_artic_df.primer_trimming.to_csv(f'{outdir}/primer_trimming_artic.sh', index=False, header=False)
cmds_artic_df.sort_index_bam.to_csv(f'{outdir}/sort_index_bam_artic.sh', index=False, header=False)
cmds_artic_df.samtools_depth.to_csv(f'{outdir}/samtools_depth_artic.sh', index=False, header=False)
cmds_artic_df.mpileup_variants.to_csv(f'{outdir}/mpileup_variants_artic.sh', index=False, header=False)
cmds_artic_df.freebayes_variants.to_csv(f'{outdir}/freebayes_artic.sh', index=False, header=False)
cmds_artic_df.picard_readgrp.to_csv(f'{outdir}/picard_readgrp_artic.sh', index=False, header=False)
cmds_artic_df.index_bam.to_csv(f'{outdir}/index_bam_artic.sh', index=False, header=False)
cmds_artic_df.gatk_variants.to_csv(f'{outdir}/gatk_artic.sh', index=False, header=False)

In [86]:
# f'gatk CreateSequenceDictionary -R {scv2_ref}'

'gatk-launch CreateSequenceDictionary -R /groups/banfield/projects/industrial/wastewater_bayarea/2021/mapping/scv2.fasta'

# Pipeline - Swift

### Swift

In [180]:
cmds = []
for row in reads_swift_df.itertuples():
    r1 = row.read1
    r2 = row.read2
    s = row.sample
    trimR = f'{bb} -Xmx5g k=23 mink=11 hdist=1 tbo tpe t=2 ref={adapters} ktrim=r ftm=5 -in1={rawreads_sw}/{r1} -in2={rawreads_sw}/{r2} -out1={rawreads_sw}/{s}.1.noadapt.fastq.gz -out2={rawreads_sw}/{s}.2.noadapt.fastq.gz'
    #filtphiX = f'{bb} -Xmx5g t=2 k=31 hdist=1 -in1={rawreads}/{s}.1.noadapt.fastq.gz -in2={rawreads}/{s}.2.noadapt.fastq.gz out1={rawreads}/{s}.1.nophix.fastq.gz out2={rawreads}/{s}.2.nophix.fastq.gz ref={phiX} stats={s}_stats_phix.txt'
    qtrim = f'sickle pe -l 75 -f {rawreads_sw}/{s}.1.noadapt.fastq.gz -r {rawreads_sw}/{s}.2.noadapt.fastq.gz -t sanger -o {trimmedreads_sw}/{s}.PE.1.fastq -p {trimmedreads_sw}/{s}.PE.2.fastq -s {trimmedreads_sw}/{s}.SR.fastq'
    clean = f'rm {rawreads_sw}/{s}.1.noadapt.fastq.gz {rawreads_sw}/{s}.2.noadapt.fastq.gz {trimmedreads_sw}/{s}.SR.fastq'
    #qc = f'{fastqc} -t 2 {s}.PE.1.fastq {s}.PE.2.fastq'
    gz1 = f'pigz -p 2 {trimmedreads_sw}/{s}.PE.1.fastq'
    gz2 = f'pigz -p 2 {trimmedreads_sw}/{s}.PE.2.fastq'
    cmd = [trimR, qtrim, clean, gz1, gz2]
    cmd = "; ".join(cmd)
    cmds.append([s, cmd])

cmds_swift_df = pd.DataFrame(cmds, columns=[
                        'sample',
                        'read_trimming'
                    ])

In [114]:
cmds_swift_df.read_trimming.to_csv('/Users/rosekantor/work/wbe_sequence_analysis/read_trimming_swift.sh', index=False, header=False)

In [190]:
cmds = []
for s in reads_swift_df['sample']:
    mapping = f'bowtie2 -x {scv2_ref} -p 16 --no-unal -1 {trimmedreads_sw}/{s}.PE.1.fastq.gz -2 {trimmedreads_sw}/{s}.PE.2.fastq.gz 2>{mapping_sw}/{s}.log | samtools sort -n -O sam -o {mapping_sw}/{s}.sam'
    primer_trimming = f'{primerclip} {swift_masterfile} {mapping_sw}/{s}.sam {mapping_sw}/{s}.primer_trimmed.sam'
    #sam2bam = 
    #sort_index_bam = f'samtools sort {s}.primer_trimmed.bam > {s}.primer_trimmed.sorted.bam && samtools index {s}.primer_trimmed.sorted.bam'
    #mpileup_variants = f'samtools mpileup -aa -A  -B -Q 0 {s}.primer_trimmed.sorted.bam | singularity run docker://quay.io/biocontainers/ivar:1.3.1--h089eab3_0 ivar variants -p {s}.ivar -q 20 -t 0.03 -r scv2.fasta -g sc2_noCDS.gff'
    cmds.append([s, mapping, primer_trimming])#([s, mapping, primer_trimming, sort_index_bam, mpileup_variants])

variant_df = pd.DataFrame(cmds, columns=[
                        'sample',
                        'mapping',
                        'primer_trimming',
#                         'sort_index_bam',
#                         'mpileup_variants'
                    ])
cmds_swift_df = cmds_swift_df.merge(variant_df, on='sample')

In [191]:
cmds_swift_df.read_trimming.to_csv('/Users/rosekantor/work/wbe_sequence_analysis/read_trimming_swift.sh', index=False, header=False)
cmds_swift_df.mapping.to_csv('/Users/rosekantor/work/wbe_sequence_analysis/mapping_swift.sh', index=False, header=False)
cmds_swift_df.primer_trimming.to_csv('/Users/rosekantor/work/wbe_sequence_analysis/primer_trimming_swift.sh', index=False, header=False)